In [27]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [1]:
# Dataset: https://github.com/TrainingByPackt/Deep-Learning-for-Natural-Language-Processing/blob/master/Lesson%2003/train_comment_small_50.csv
from google.colab import files
uploaded = files.upload()

Saving train_comment_small_50.csv.txt to train_comment_small_50.csv.txt


In [28]:
!ls

sample_data  train_comment_small_50.csv  train_comment_small_50.csv.txt


In [37]:
# Neural Network to predict if review is Pos or Neg

# Colab reads csv as txt for some reason, so do this
df = pd.read_csv('train_comment_small_50.csv.txt',delimiter=',')
df.to_csv('train_comment_small_50.csv',index = None) 
df = pd.read_csv('train_comment_small_50.csv',delimiter=',')
df.head()

,comment_text,toxic
0,"""\n \n I'm sure if you worked on articles othe...",0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\n More\n I can't make any real suggestions o...",0
4,REDIRECT Talk:Voydan Pop Georgiev- Chernodrinski,0


In [38]:
# Preprocess text
def text_cleaning(text):
  # HTML and other characters
  text = re.sub('<[^<]+?>',' ',text)
  text = text.replace('\\"','')
  text = text.replace('"','')
  text = text.replace('\n','')
  return text

df['cleaned_text'] = df['comment_text'].apply(text_cleaning)
df['cleaned_text'][:5]

0      I'm sure if you worked on articles other tha...
1    D'aww! He matches this background colour I'm s...
2    Hey man, I'm really not trying to edit war. It...
3     More I can't make any real suggestions on imp...
4     REDIRECT Talk:Voydan Pop Georgiev- Chernodrinski
Name: cleaned_text, dtype: object

In [39]:
# Train/Test split
X_train, x_test, y_train, y_test = train_test_split(df['cleaned_text'],
                                                    df['toxic'], test_size=0.2)

In [42]:
# Convert text into word count vectors for ML model
vectorizer = CountVectorizer(binary=True,stop_words=stopwords.words('english'),
                             lowercase=True,min_df=3,max_df=0.9, max_features=5000)
X_train_onehot = vectorizer.fit_transform(X_train)

In [43]:
# 2-layer NN
nn = Sequential()
# Hidden Layer
nn.add(Dense(units=500,activation='relu',input_dim=len(vectorizer.get_feature_names())))
# Output Layer using Sigmoid
nn.add(Dense(units=1,activation='sigmoid'))
# Compile the model and summarise
nn.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
nn.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 500)               28000     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 501       
Total params: 28,501
Trainable params: 28,501
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Train Model
nn.fit(X_train_onehot[-20:],y_train[-20:],epochs=5,batch_size=128,verbose=1,
       validation_data=(X_train_onehot[-20:],y_train[-20:]))

In [ ]:
scores - nn.evaluate(vectorizer.transform(X_test),y_test, verbose=1)
print('Accuracy:',scores[1])
model.save('nn.h5')